In [1]:
# Torch
import torch
from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler
# Data Handling
import numpy as np
import pandas as pd
# System
import os
import pprint
import itertools
# Custom
import models
import datasets
from helpers import utils

In [2]:
########################## FILE SYSTEM SETTINGS ###########################

savedir_base = "/home/jovyan/work/runs/LCFCN"
datadir = "/home/jovyan/work/DENMARK/250x250"

####################### EXPERIMENT SETTINGS ################################

experiment_id = "cob_test"
run_path = os.path.join(savedir_base, experiment_id)
exp_dict = utils.loadJSON(os.path.join(run_path, 'exp_dict.json'))

pprint.pprint(exp_dict)

{'batch_size': 1,
 'dataset': {'name': 'denmark', 'transform': 'rgb_normalize'},
 'dataset_size': {'train': 'all', 'val': 'all'},
 'lr': 1e-05,
 'max_epoch': 5,
 'model': {'base': 'fcn8_vgg16', 'name': 'cob'},
 'optimizer': 'adam'}


In [3]:
dataset_size = {'test': 10}
batch_size = 1

test_set = datasets.getDataset(dataset = exp_dict["dataset"]["name"],
                               split = "test",
                               datadir = datadir,
                               transform = exp_dict['dataset']['transform'],
                               dataset_size = dataset_size)

test_sampler = torch.utils.data.SequentialSampler(test_set)

test_loader = DataLoader(test_set,
                         sampler = test_sampler,
                         batch_size = batch_size,
                         num_workers = 2)

print("Testing Data loaded...")

Testing Data loaded...


In [4]:
n_classes = test_set.n_classes
model = models.getModel(exp_dict, n_classes).cuda()

run_model = utils.loadTorch(os.path.join(run_path, 'model_best.pth'))
model.loadStateDict(run_model)

print("Model loaded...")

Model loaded...


In [67]:
from models.losses import cob_loss
stop = True

with torch.no_grad():
    for i, batch in enumerate(test_loader):
        if stop:
            # Declare Eval Mode
            model.eval()

            # Load Data to GPU
            images = batch["images"].cuda()
            points = batch["points"].long().cuda()
            shapes = batch["shapes"]

            # Forward Prop
            logits = model.model_base.forward(images)

            # Sigmoid Function
            probs = logits.sigmoid().cpu().numpy()
            blobs = cob_loss.getBlobs(probs=probs)
            
            points_numpy = points.cpu().numpy()
            point_blobs = np.unique(blobs * points_numpy)
            
            if (np.unique(point_blobs) !=0 ).sum() > 1:
                blob_mask1 = blobs==point_blobs[1]
                blob_mask2 = blobs==point_blobs[2]
                stop = False

In [68]:
from PIL import Image, ImageDraw

flat_list1 = [item.item() for sublist in shapes[0] for item in sublist]
flat_list2 = [item.item() for sublist in shapes[1] for item in sublist]
flat_list3 = [item.item() for sublist in shapes[2] for item in sublist]

img = Image.new('L', (250, 250), 0)
ImageDraw.Draw(img).polygon(flat_list1, outline=1, fill=1)
ImageDraw.Draw(img).polygon(flat_list2, outline=1, fill=1)
ImageDraw.Draw(img).polygon(flat_list3, outline=1, fill=1)

In [69]:
from helpers import haven_viz
img_src = haven_viz.get_image(batch["images"], denorm="rgb")

img_pred = haven_viz.mask_on_image(img_src, blobs.squeeze())
img_pred_shape = haven_viz.mask_on_image(img_src, np.array(img).squeeze())

In [70]:
haven_viz.save_image('test.jpg', img_pred)
haven_viz.save_image('test_shapes.jpg', img_pred_shape)

In [61]:
import sys

np.set_printoptions(threshold=sys.maxsize)
np.array(img)

250

In [65]:
len(shapes)

3